In [23]:
from pyspark.sql import functions as F

def create_gold_analytics_tables(silver_catalog: str):
    """
    Crée les tables analytiques Gold à partir de la couche Silver.
    Compatible Spark + Iceberg, corrigé pour les booléens et tip_percentage.
    """
    print("\n📊 Création des tables analytiques Gold...")

    # Charger la table silver complète
    trips = spark.table(f"{silver_catalog}.silver.trips_complete")

    # Ajouter tip_percentage si absent
    if "tip_percentage" not in trips.columns:
        trips = trips.withColumn(
            "tip_percentage",
            F.when(F.col("total_amount") > 0, F.col("tip_amount") / F.col("total_amount") * 100).otherwise(0)
        )

    # Ajouter pickup_hour et pickup_date si absent
    if "pickup_hour" not in trips.columns:
        trips = trips.withColumn("pickup_hour", F.hour(F.col("tpep_pickup_datetime")))
    if "pickup_date" not in trips.columns:
        trips = trips.withColumn("pickup_date", F.to_date(F.col("tpep_pickup_datetime")))

    # =====================================================
    # 1️⃣ TABLE GOLD: DAILY METRICS
    # =====================================================
    print("   📅 Création de daily_metrics...")

    daily_metrics = (
        trips.groupBy("pickup_date")
        .agg(
            F.count("*").alias("total_trips"),
            F.sum("total_amount").alias("daily_revenue"),
            F.avg("trip_distance").alias("avg_trip_distance"),
            F.avg("trip_duration_minutes").alias("avg_trip_duration"),
            F.avg("passenger_count").alias("avg_passenger_count"),
            F.sum("tip_amount").alias("total_tips"),
            F.avg("tip_percentage").alias("avg_tip_percentage"),
            F.countDistinct("VendorID").alias("active_vendors"),
            F.sum(F.when(F.col("payment_type") == 1, 1).otherwise(0)).alias("credit_card_trips"),
            F.sum(F.when(F.col("payment_type") == 2, 1).otherwise(0)).alias("cash_trips"),
            F.sum(F.when(F.col("is_rainy") == 1, 1).otherwise(0)).alias("rainy_trips")
        )
        .withColumn(
            "credit_card_percentage",
            F.when(F.col("total_trips") > 0, F.col("credit_card_trips") / F.col("total_trips") * 100)
        )
        .withColumn(
            "revenue_per_trip",
            F.col("daily_revenue") / F.col("total_trips")
        )
        .withColumn("year", F.year("pickup_date"))
        .withColumn("month", F.month("pickup_date"))
    )

    daily_metrics.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{silver_catalog}.gold.daily_metrics")

    print(f"   ✅ Table gold.daily_metrics créée: {daily_metrics.count():,} lignes")

    # =====================================================
    # 2️⃣ TABLE GOLD: HOURLY PATTERNS
    # =====================================================
    print("   🕐 Création de hourly_patterns...")

    hourly_patterns = trips.groupBy("pickup_date", "pickup_hour") \
        .agg(
            F.count("*").alias("trip_count"),
            F.avg("trip_distance").alias("avg_distance"),
            F.avg("trip_duration_minutes").alias("avg_duration"),
            F.avg("total_amount").alias("avg_fare"),
            F.avg("speed_mph").alias("avg_speed"),
            F.avg("tip_percentage").alias("avg_tip_pct"),
            F.sum(F.when(F.col("is_weekend") == 1, 1).otherwise(0)).alias("weekend_trips"),
            F.sum(F.when(F.col("is_weekend") == 0, 1).otherwise(0)).alias("weekday_trips")
        ) \
        .withColumn(
            "hour_type",
            F.when((F.col("pickup_hour") >= 6) & (F.col("pickup_hour") <= 9), "morning_rush")
             .when((F.col("pickup_hour") >= 17) & (F.col("pickup_hour") <= 20), "evening_rush")
             .when((F.col("pickup_hour") >= 22) | (F.col("pickup_hour") <= 4), "night")
             .otherwise("regular_hours")
        ) \
        .withColumn("year", F.year("pickup_date")) \
        .withColumn("month", F.month("pickup_date"))

    hourly_patterns.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{silver_catalog}.gold.hourly_patterns")

    print(f"   ✅ Table gold.hourly_patterns créée: {hourly_patterns.count():,} lignes")

    # =====================================================
    # 3️⃣ TABLE GOLD: TOP ROUTES
    # =====================================================
    print("   🗺️  Création de top_routes...")

    trips_with_route = trips.withColumn(
        "route_id",
        F.concat(F.col("PULocationID"), F.lit("_"), F.col("DOLocationID"))
    )

    top_routes = trips_with_route.groupBy("route_id", "PULocationID", "DOLocationID", "pickup_date") \
        .agg(
            F.count("*").alias("trip_count"),
            F.avg("trip_distance").alias("avg_distance"),
            F.avg("trip_duration_minutes").alias("avg_duration"),
            F.avg("total_amount").alias("avg_fare"),
            F.avg("speed_mph").alias("avg_speed"),
            F.avg("tip_percentage").alias("avg_tip_pct"),
            F.sum("passenger_count").alias("total_passengers")
        ) \
        .withColumn(
            "efficiency_ratio",
            F.col("avg_distance") / F.when(F.col("avg_duration") > 0, F.col("avg_duration")).otherwise(1)
        ) \
        .withColumn("year", F.year("pickup_date")) \
        .withColumn("month", F.month("pickup_date")) \
        .orderBy(F.desc("trip_count")) \
        .limit(1000)

    top_routes.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{silver_catalog}.gold.top_routes")

    print(f"   ✅ Table gold.top_routes créée: {top_routes.count():,} lignes")

    # =====================================================
    # 4️⃣ TABLE GOLD: DRIVER PERFORMANCE
    # =====================================================
    print("   🚕 Création de driver_performance...")

    driver_performance = trips.groupBy("VendorID", "pickup_date") \
        .agg(
            F.count("*").alias("trips_completed"),
            F.sum("total_amount").alias("total_revenue"),
            F.avg("trip_duration_minutes").alias("avg_trip_duration"),
            F.avg("total_amount").alias("avg_fare"),
            F.avg("speed_mph").alias("avg_speed"),
            F.sum("tip_amount").alias("total_tips"),
            F.avg("tip_percentage").alias("avg_tip_percentage"),
            F.sum(F.when(F.col("payment_type") == 1, 1).otherwise(0)).alias("credit_card_trips"),
            F.sum(F.when(F.col("payment_type") == 2, 1).otherwise(0)).alias("cash_trips")
        ) \
        .withColumn(
            "revenue_per_hour",
            F.when(F.col("avg_trip_duration") > 0,
                   F.col("total_revenue") / (F.col("avg_trip_duration") * F.col("trips_completed") / 60))
        ) \
        .withColumn(
            "credit_card_percentage",
            (F.col("credit_card_trips") / F.col("trips_completed")) * 100
        ) \
        .withColumn("year", F.year("pickup_date")) \
        .withColumn("month", F.month("pickup_date"))

    driver_performance.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{silver_catalog}.gold.driver_performance")

    print(f"   ✅ Table gold.driver_performance créée: {driver_performance.count():,} lignes")

    # =====================================================
    # 5️⃣ TABLE GOLD: WEATHER IMPACT
    # =====================================================
    print("   🌤️  Création de weather_impact...")

    weather_impact = trips.groupBy("pickup_date", "is_rainy", "is_cold") \
        .agg(
            F.count("*").alias("total_trips"),
            F.avg("trip_distance").alias("avg_distance"),
            F.avg("trip_duration_minutes").alias("avg_duration"),
            F.avg("total_amount").alias("avg_fare"),
            F.avg("speed_mph").alias("avg_speed"),
            F.avg("tip_percentage").alias("avg_tip_pct"),
            F.avg("temp").alias("avg_temp"),
            F.avg("rhum").alias("avg_rhum"),
            F.avg("prcp").alias("avg_prcp"),
            F.avg("wspd").alias("avg_wspd"),
            F.avg("pres").alias("avg_pres")
        ) \
        .withColumn(
            "weather_condition",
            F.when((F.col("is_rainy") == 1) & (F.col("is_cold") == 1), "rainy_cold")
             .when((F.col("is_rainy") == 1) & (F.col("is_cold") == 0), "rainy_warm")
             .when((F.col("is_rainy") == 0) & (F.col("is_cold") == 1), "dry_cold")
             .otherwise("dry_warm")
        ) \
        .withColumn("year", F.year("pickup_date")) \
        .withColumn("month", F.month("pickup_date"))

    weather_impact.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{silver_catalog}.gold.weather_impact")

    print(f"   ✅ Table gold.weather_impact créée: {weather_impact.count():,} lignes")

    # =====================================================
    # 6️⃣ TABLE GOLD: PAYMENT ANALYSIS
    # =====================================================
    print("   💳 Création de payment_analysis...")

    payment_analysis = trips.groupBy("pickup_date", "payment_type") \
        .agg(
            F.count("*").alias("trip_count"),
            F.sum("total_amount").alias("total_revenue"),
            F.avg("total_amount").alias("avg_fare"),
            F.sum("tip_amount").alias("total_tips"),
            F.avg("tip_percentage").alias("avg_tip_pct"),
            F.avg("trip_distance").alias("avg_distance")
        ) \
        .withColumn(
            "payment_type_label",
            F.when(F.col("payment_type") == 1, "credit_card")
             .when(F.col("payment_type") == 2, "cash")
             .when(F.col("payment_type") == 3, "no_charge")
             .when(F.col("payment_type") == 4, "dispute")
             .when(F.col("payment_type") == 5, "unknown")
             .when(F.col("payment_type") == 6, "voided")
             .otherwise("other")
        ) \
        .withColumn("year", F.year("pickup_date")) \
        .withColumn("month", F.month("pickup_date"))

    payment_analysis.write \
        .format("iceberg") \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .saveAsTable(f"{silver_catalog}.gold.payment_analysis")

    print(f"   ✅ Table gold.payment_analysis créée: {payment_analysis.count():,} lignes")

    print("\n🎉 PIPELINE GOLD TERMINÉ AVEC SUCCÈS!")

    return {
        "daily_metrics": daily_metrics,
        "hourly_patterns": hourly_patterns,
        "top_routes": top_routes,
        "driver_performance": driver_performance,
        "weather_impact": weather_impact,
        "payment_analysis": payment_analysis
    }


In [24]:
create_gold_analytics_tables(silver_catalog="iceberg")


📊 Création des tables analytiques Gold...
   📅 Création de daily_metrics...


   ✅ Table gold.daily_metrics créée: 374 lignes
   🕐 Création de hourly_patterns...


   ✅ Table gold.hourly_patterns créée: 8,796 lignes
   🗺️  Création de top_routes...


   ✅ Table gold.top_routes créée: 1,000 lignes
   🚕 Création de driver_performance...


   ✅ Table gold.driver_performance créée: 740 lignes
   🌤️  Création de weather_impact...


   ✅ Table gold.weather_impact créée: 632 lignes
   💳 Création de payment_analysis...


[Stage 107:=======================================>               (10 + 2) / 14]

   ✅ Table gold.payment_analysis créée: 1,476 lignes

🎉 PIPELINE GOLD TERMINÉ AVEC SUCCÈS!


{'daily_metrics': DataFrame[pickup_date: date, total_trips: bigint, daily_revenue: double, avg_trip_distance: double, avg_trip_duration: double, avg_passenger_count: double, total_tips: double, avg_tip_percentage: double, active_vendors: bigint, credit_card_trips: bigint, cash_trips: bigint, rainy_trips: bigint, credit_card_percentage: double, revenue_per_trip: double, year: int, month: int],
 'hourly_patterns': DataFrame[pickup_date: date, pickup_hour: int, trip_count: bigint, avg_distance: double, avg_duration: double, avg_fare: double, avg_speed: double, avg_tip_pct: double, weekend_trips: bigint, weekday_trips: bigint, hour_type: string, year: int, month: int],
 'top_routes': DataFrame[route_id: string, PULocationID: int, DOLocationID: int, pickup_date: date, trip_count: bigint, avg_distance: double, avg_duration: double, avg_fare: double, avg_speed: double, avg_tip_pct: double, total_passengers: bigint, efficiency_ratio: double, year: int, month: int],
 'driver_performance': DataF

In [26]:
spark.table("iceberg.gold.daily_metrics").limit(5).toPandas()

,pickup_date,total_trips,daily_revenue,avg_trip_distance,avg_trip_duration,avg_passenger_count,total_tips,avg_tip_percentage,active_vendors,credit_card_trips,cash_trips,rainy_trips,credit_card_percentage,revenue_per_trip,year,month
0,2024-04-20,107727,2899354.92,3.129840,16.028695,1.434775,366879.91,12.285536,2,90509,15774,23553,84.017006,26.913911,2024,4
1,2024-04-08,85102,2493977.18,3.741918,15.358545,1.315762,311888.44,12.163303,2,70852,13235,0,83.255388,29.305741,2024,4
2,2024-04-02,105496,3027449.40,3.223729,17.765241,1.327083,379606.31,12.312452,2,88768,15499,82911,84.143475,28.697291,2024,4
3,2024-04-06,102947,2727436.14,3.116937,15.333505,1.464491,342324.17,12.214231,2,85948,15643,0,83.487620,26.493595,2024,4
4,2024-04-27,103543,2776765.38,3.215236,15.379805,1.456149,347605.29,12.200026,2,86180,15984,0,83.231121,26.817509,2024,4
